In [2]:
import os
import tensorflow as tf
#from object_detection.utils import visualization_utils as viz_utils
from object_detection.builders import model_builder
from object_detection.utils import config_util
from PIL import Image
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import cv2

plates_config_dir = r"C:\Users\PC\Documents\projects\carPlates\models\platesModel\pipeline.config"
plates_ckpt_dir = r"C:\Users\PC\Documents\projects\carPlates\models\platesModel\ckpt-43"

letters_config_dir = r"C:\Users\PC\Documents\projects\carPlates\models\lettersModel\pipeline.config"
letters_ckpt_dir = r"C:\Users\PC\Documents\projects\carPlates\models\lettersModel\ckpt-101"

plates_configs = config_util.get_configs_from_pipeline_file(plates_config_dir)
plates_detection_model = model_builder.build(model_config=plates_configs['model'], is_training=False)

letters_configs = config_util.get_configs_from_pipeline_file(letters_config_dir)
letters_detection_model = model_builder.build(model_config=letters_configs['model'], is_training=False)

# Restore checkpoint
plates_ckpt = tf.compat.v2.train.Checkpoint(model=plates_detection_model)
plates_ckpt.restore(plates_ckpt_dir).expect_partial()


# Restore checkpoint
letters_ckpt = tf.compat.v2.train.Checkpoint(model=letters_detection_model)
letters_ckpt.restore(letters_ckpt_dir).expect_partial()



In [2]:
def detect_fn(image, detection_model):

    image, shapes = detection_model.preprocess(image)
    prediction_dict = detection_model.predict(image, shapes)
    detections = detection_model.postprocess(prediction_dict, shapes)

    return detections

In [3]:
def detect_img(img, detection_model):
    image_np = np.array(img)
    input_tensor = tf.convert_to_tensor(np.expand_dims(image_np, 0), dtype=tf.float32)

    detections = detect_fn(input_tensor, detection_model)

        # All outputs are batches tensors.
        # Convert to numpy arrays, and take index [0] to remove the batch dimension.
        # We're only interested in the first num_detections.
    num_detections = int(detections.pop('num_detections'))
    detections = {key: value[0, :num_detections].numpy()
                      for key, value in detections.items()}
    detections['num_detections'] = num_detections

        # detection_classes should be ints.
    detections['detection_classes'] = detections['detection_classes'].astype(np.int64)

    label_id_offset = 1
    image_np_with_detections = image_np.copy()
    
    
    output_detections = []
    
    for i in range(len(detections['detection_boxes'])):
        if detections['detection_scores'][i] > 0.4:
            output_detections.append(("letter", detections['detection_boxes_strided'][i]))
    
    
    viz_utils.visualize_boxes_and_labels_on_image_array(
                image_np_with_detections,
                detections['detection_boxes'],
                detections['detection_classes']+label_id_offset,
                detections['detection_scores'],
                {0 : "1"},
                use_normalized_coordinates=True,
                max_boxes_to_draw=200,
                min_score_thresh=.4,
                agnostic_mode=False,
                line_thickness = 5,
                #skip_labels=True,
                #skip_scores=True
        )
    return Image.fromarray(image_np_with_detections)

In [4]:
def return_boxes(img, detection_model):
    shape = img.shape
    
    input_tensor = tf.convert_to_tensor(np.expand_dims(img, 0), dtype=tf.float32)

    detections = detect_fn(input_tensor, detection_model)

    num_detections = int(detections.pop('num_detections'))
    detections = {key: value[0, :num_detections].numpy()
                      for key, value in detections.items()}
    
    output_boxes = []
    for i in range(len(detections['detection_scores'])):
        if detections['detection_scores'][i] > 0.4:
            output_boxes.append(detections['detection_boxes'][i])
    if len(output_boxes) > 0:
        output_boxes = np.array(output_boxes)
        output_boxes[:, 0] *= shape[0]
        output_boxes[:, 1] *= shape[1]
        output_boxes[:, 2] *= shape[0]
        output_boxes[:, 3] *= shape[1]
        output_boxes = np.array(output_boxes).astype(int)
    # shape as [ymin, xmin, ymax, xmax]
    return output_boxes

In [5]:
from tensorflow.keras.models import load_model
classification_model = load_model("models/letterModel_mod")

In [6]:
df = pd.read_csv("data/letters420/letter_labels.csv")
df = df.dropna()
lst = []
for i in range(len(df)):
    if df['label'][i] == '0':
        lst.append(i)
df = df.drop(lst, axis = 0)
s = list(set(df['label']))
s = sorted(s)
dic = {i:s[i] for i in range(len(s))}

In [7]:
def get_rotated_img(img):
        img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        edges = cv2.Canny(img_gray, 100, 200, None, 3)
        lines = cv2.HoughLines(edges, 1, np.pi / 180, 100)
        total_theta = []
        if lines is not None:
            for i in range(len(lines)):
                if (abs(lines[i][0][1]) < 0.45 or abs(lines[i][0][1] - np.pi) < 0.45):
                    continue
                else:
                    total_theta.append(lines[i][0][1])
            if len(total_theta) != 0:
                avg_theta = sum([theta for theta in total_theta])/len(total_theta) * (180/np.pi) - 90
            else:
                avg_theta = 0
            (h, w) = img.shape[:2]
            (cX, cY) = (w // 2, h // 2)
            M = cv2.getRotationMatrix2D((cX, cY), avg_theta, 1.0)
            rotated = cv2.warpAffine(img, M, (w, h))
            return rotated
        image = np.array(img_gray)
        return img

In [ ]:
def get_bin(img):
    mx, mn = (img[0][0], img[0][0])
    for row in img:
        rmax = max(row)
        rmin = min(row)
        if rmax > mx:
            mx = rmax
        if rmin < mn:
            mn = rmin
    new_img = []
    mn = int(mn)
    mx = int(mx)
    thresh = (mn + mx)/2
    for row in img:
        nrow = []
        for pixel in row:
            if pixel >= thresh:
                nrow.append(255)
            else:
                nrow.append(0)
        new_img.append(nrow)
    return np.array(new_img)

In [8]:
def put_letters_together(lst):
    output = ""
    for letter in lst:
        output += letter + " "
    return output

In [9]:
from PIL import Image, ImageFont, ImageDraw
import time

In [10]:
raw_df = pd.read_csv("data/on1000/all_train_mod.csv")
directories = list(set(raw_df['filename']))
raw_dir = r"E:\DataSets\carPlatesDataset\raw"
all_dir = [os.path.join(raw_dir, item) for item in os.listdir(raw_dir)]
all_dir

['E:\\DataSets\\carPlatesDataset\\raw\\IMG_20220330_121505.jpg',
 'E:\\DataSets\\carPlatesDataset\\raw\\IMG_20220330_121509.jpg',
 'E:\\DataSets\\carPlatesDataset\\raw\\IMG_20220330_121518.jpg',
 'E:\\DataSets\\carPlatesDataset\\raw\\IMG_20220330_121656.jpg',
 'E:\\DataSets\\carPlatesDataset\\raw\\IMG_20220330_121658.jpg',
 'E:\\DataSets\\carPlatesDataset\\raw\\IMG_20220330_121704.jpg',
 'E:\\DataSets\\carPlatesDataset\\raw\\IMG_20220330_121706.jpg',
 'E:\\DataSets\\carPlatesDataset\\raw\\IMG_20220330_121708.jpg',
 'E:\\DataSets\\carPlatesDataset\\raw\\IMG_20220330_121711.jpg',
 'E:\\DataSets\\carPlatesDataset\\raw\\IMG_20220330_122048.jpg',
 'E:\\DataSets\\carPlatesDataset\\raw\\IMG_20220330_122115.jpg',
 'E:\\DataSets\\carPlatesDataset\\raw\\IMG_20220330_122117.jpg',
 'E:\\DataSets\\carPlatesDataset\\raw\\IMG_20220330_122234.jpg',
 'E:\\DataSets\\carPlatesDataset\\raw\\IMG_20220330_122238.jpg',
 'E:\\DataSets\\carPlatesDataset\\raw\\IMG_20220330_122241.jpg',
 'E:\\DataSets\\carPlates

In [11]:
len(directories)

998

In [12]:
len(all_dir)

1409

In [13]:
testing_dir = []
for item in all_dir:
    if item not in directories:
        testing_dir.append(item)
len(testing_dir)

411

In [17]:
start_time = time.time()
img_path = r"C:\Users\PC\OneDrive\Desktop\test3.jpg"
#img_path = testing_dir[398]
if img_path in directories:
    print("INSIDE TRAINING DIRECTORY")
img = Image.open(img_path)
#img = img.rotate(180)
img = np.array(img)
start_time = time.time()
plate_boxes = return_boxes(img, plates_detection_model)

plate_imgs = []
letter_boxes = []

for plate_box in plate_boxes:
    plate_img = img[plate_box[0]: plate_box[2], plate_box[1]:plate_box[3]]
    plate_img = get_rotated_img(plate_img)
    plate_imgs.append(plate_img)
    letter_boxes.append(return_boxes(plate_img, letters_detection_model))

    
output_predictions = []
#letter_imgs = []
for i in range(len(letter_boxes)):
    plate_number = []
    sorting_list = []
    for letter_box in letter_boxes[i]:
        letter_img = Image.fromarray(plate_imgs[i][letter_box[0]:letter_box[2], letter_box[1]:letter_box[3]])
        letter_img = letter_img.convert("L")
        letter_img = np.array([np.array(letter_img.resize((96, 96)))/255])
        prediction = np.argmax(classification_model.predict(letter_img))
        prediction = dic[prediction]
        plate_number.append(prediction)
        sorting_list.append(letter_box[1])
    plate_number = [num for _, num in sorted(zip(sorting_list, plate_number))]
    #plate_number.reverse()
    output_predictions.append(plate_number)


    
print(output_predictions)
#for i in range(len(letter))
fontpath = "font/arial.ttf" # <== download font
font = ImageFont.truetype(fontpath, 48)
img_pil = Image.fromarray(img)
draw = ImageDraw.Draw(img_pil)

final_img = Image.fromarray(img)
draw = ImageDraw.Draw(img_pil)
for i in range(len(output_predictions)):
    x_min = plate_boxes[i][1]
    y_min = plate_boxes[i][0]
    font = ImageFont.truetype(fontpath, int((plate_boxes[i][3]-plate_boxes[i][1])/5))
    draw.text((x_min, y_min),put_letters_together(output_predictions[i]), font = font)
end_time = time.time()
print(f"process took {end_time - start_time} seconds")
img_pil.show()
#cv2.imshow('test', np.array(img_pil.resize((1280, 720))))
#cv2.waitKey(0)

1/1 [==============================] - 0s 25ms/step
[['م']]
process took 0.27324724197387695 seconds
